In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

# load the Tiny Shakespeare dataset
dataset, info = tfds.load('tiny_shakespeare', with_info=True, as_supervised=False)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/tiny_shakespeare/incomplete.N441J3_1.0.0/tiny_shakespeare-train.tfrecord*.…

Generating validation examples...:   0%|          | 0/1 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/tiny_shakespeare/incomplete.N441J3_1.0.0/tiny_shakespeare-validation.tfrec…

Generating test examples...:   0%|          | 0/1 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/tiny_shakespeare/incomplete.N441J3_1.0.0/tiny_shakespeare-test.tfrecord*..…

Dataset tiny_shakespeare downloaded and prepared to /root/tensorflow_datasets/tiny_shakespeare/1.0.0. Subsequent calls will reuse this data.


In [ ]:
# get the text from the dataset
text = next(iter(dataset['train']))['text'].numpy().decode('utf-8')

# create a mapping from unique characters to indices
vocab = sorted(set(text))
char2idx = {char: idx for idx, char in enumerate(vocab)}
idx2char = np.array(vocab)

# numerically represent the characters
text_as_int = np.array([char2idx[c] for c in text])

# create training examples and targets
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

# create training sequences
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
# batch size and buffer size
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

In [ ]:
# length of the vocabulary
vocab_size = len(vocab)

# the embedding dimension
embedding_dim = 256

# number of RNN units
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [ ]:
import os

# directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'

# name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

# train the model
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
155/155 [==============================] - 17s 70ms/step - loss: 2.6216
Epoch 2/20
155/155 [==============================] - 12s 70ms/step - loss: 1.9141
Epoch 3/20
155/155 [==============================] - 13s 73ms/step - loss: 1.6613
Epoch 4/20
155/155 [==============================] - 12s 70ms/step - loss: 1.5240
Epoch 5/20
155/155 [==============================] - 13s 70ms/step - loss: 1.4382
Epoch 6/20
155/155 [==============================] - 13s 71ms/step - loss: 1.3795
Epoch 7/20
155/155 [==============================] - 13s 72ms/step - loss: 1.3321
Epoch 8/20
155/155 [==============================] - 13s 71ms/step - loss: 1.2915
Epoch 9/20
155/155 [==============================] - 12s 70ms/step - loss: 1.2537
Epoch 10/20
155/155 [==============================] - 12s 70ms/step - loss: 1.2167
Epoch 11/20
155/155 [==============================] - 12s 70ms/step - loss: 1.1806
Epoch 12/20
155/155 [==============================] - 12s 70ms/step - loss: 1.1415
E

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
def generate_text(model, start_string):
    num_generate = 1000

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

print(generate_text(model, start_string=u"QUEEN: So, lets end this"))

QUEEN: So, lets end this rg, and now
Charge hath the projects of denies; means to make
What offer manner to that makes your
pease, for the rest, and this be made?

JULIET:
Ay, timple, here theseretted to the scene.

JOHN OF GAUNT:
I now rather hold rich in bown, and but the ground of kings
In Brhand Bepake the Duke of Gloucester,
Or who be sought for your own soul,
And never bring out my kinsman coars.

WARWICK:
To whom, Catesby; York! and so shall we see
The common means to wait upon it, marry, 'tis more chance to dead my cast
ind a more
ever such as is this absence.

ANGELO:
Methinks 'tis that have madest to do so guilty
To Bolingbroke is senators in my part.

HAMILINUS:
You shall have
mercy that of Edward's life?

ISABELLA:
The people draw our swiftless valuaps quite with heaven with your enemy, which I dream
To answer'd; I speak again, steal on the rest,
Are sticers of my numbers, I am that
Your landeding.
Go you, my lord, it was a morth before him:
Bestride the truth, nor a bed pr